In [186]:
# Libraries
import numpy as np   
import time

# Definitions
#A,b = np.array([[1.01,1.03],[0.97,1]]), np.array([2.01,1.99])
A,b = np.array([[10,7,8,7],[7,6,7,5],[8,6,10,9],[7,5,9,10]]), np.array([32,23,33,31])
alpha = 0.01
M = A + alpha*np.identity(b.shape[0])
it = 100
error = 1e-6
print np.linalg.eig(A)

(array([ 30.65091342,   3.91584018,   1.        ,   0.4332464 ]), array([[ 0.52315419,  0.59657159,  0.66712438, -0.5020158 ],
       [ 0.40738601,  0.42145656, -0.66712438,  0.78713917],
       [ 0.54477282, -0.27572485, -0.29649973, -0.28192564],
       [ 0.51338963, -0.624862  ,  0.14824986,  0.22115606]]))


In [168]:
# GMRes without optimization
def gmres(A,b,it=100, tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b.shape[0]))
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        w = np.dot(A,Q[:,i])
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
                
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
        
    return np.dot(Q[:,:i+1], y), i

In [169]:
# A better implementation of preconditioned GMRes without Cauchy integral (left)
def left_prec_gmres(A_0,b_0,M_0,it=100,tol=1e-5):
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    # copy
    A = np.copy(A_0)
    b = np.copy(b_0)
    M = np.copy(M_0)
    
    # Left preconditioner -> A and b changes
    b = gmres(M,b,it=it,tol=tol)[0]
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        w = gmres(M, np.dot(A,Q[:,i]), it=it, tol=tol)[0]
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
        
    return np.dot(Q[:,:i+1], y), i

In [170]:
# A better implementation of preconditioned GMRes without Cauchy integral (right)
def right_prec_gmres(A,b,M,it=100,tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b.shape[0]))
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        x = gmres(M, Q[:,i],it=it,tol=tol)[0]
        w = np.dot(A, x)
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    return gmres(M, x_tild,it=it,tol=tol)[0], i

In [187]:
testcases = ['normal','cancer','left','right','']
test = testcases[4]

if test == 'normal':
    print gmres(A,b,tol=error)
elif test == 'cancer':
    print paper_bad_prec_gmres(A,b,M)
elif test == 'right':
    print right_prec_gmres(A,b,M,tol=error)
elif test == 'left':
    print left_prec_gmres(A,b,M,tol=error)
else:
    print gmres(A,b,tol=error, it=it)
    print left_prec_gmres(A,b,M,tol=error, it=it)
    print right_prec_gmres(A,b,M,tol=error, it=it)
    print np.linalg.solve(A,b)

(array([ 2.57325369, -1.60931172,  1.6499896 ,  0.61847124]), 20)
(array([ 2.5890853 , -1.63480406,  1.65973646,  0.61273689]), 3)
(array([ 2.58184463, -1.61949343,  1.64255565,  0.62420738]), 3)
[ 2.57692308 -1.61538462  1.65384615  0.61538462]


In [172]:
def trapezoid2(myfun, N, a, b):
    x = np.linspace(a, b, N+1) # We want N bins, so N+1 points  
    h = x[1]-x[0]
    xmiddle = x[1:-1]
    int_val = 0
    for i in xmiddle:
        int_val += myfun(i)
    int_val = myfun(0) + 2*int_val + myfun(x[-1])
    return 0.5*h*int_val

def z(t, c, r):
    return c + r*np.complex(np.cos(t), np.sin(t))

def dz(t, r):
    return r*np.complex(np.cos(t), np.sin(t))

def g(t,l,L,M,v,f, tol=1e-6):
    fz = f(z(t, (l+L)/2., (L-l)/2.))
    dzz = dz(t, (L-l)/2.)
    p = 1./(fz*dzz)
    gmr,i = gmres(1./dzz *np.identity(v.shape[0]) - p*M, v, tol=tol)
    return gmr


def cauchy_integral(l, L, M, v, Nf = 20, N=50):
    f = lambda x: (1. - (alpha/x)**(Nf+1))/(x - alpha)
    g1 = lambda t: g(t,l,L,M,v,f)
    val = trapezoid2(g1, N, 0, 2.*np.pi) / (2.*np.pi)
    return val

In [173]:
# GMRes using contour integral to compute the preconditioner
def cauchy_prec_gmres(A,b,M,it=100,tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b.shape[0]))
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    lamb, vect = np.linalg.eig(A)
    L = np.amax(np.real(lamb))
    l = np.amin(np.real(lamb))
    d = np.amax(np.imag(lamb))
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        x = cauchy_integral(l, L, M, Q[:,i])
        w = np.dot(A, x)
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    x = cauchy_integral(l, L, M, x_tild)
    return x,i

In [184]:
# GMRes using contour integral to compute the preconditioner
def cauchy_left_prec_gmres(A_0,b_0,M_0,it=100,tol=1e-5):
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    A = np.copy(A_0)
    b = np.copy(b_0)
    M = np.copy(M_0)
    
    lamb, vect = np.linalg.eig(A)
    L = np.amax(np.real(lamb))
    l = np.amin(np.real(lamb))
    d = np.amax(np.imag(lamb))
    
    b = cauchy_integral(l, L, M, b)
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        w = cauchy_integral(l,L,M,np.dot(A,Q[:,i]))
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
    
    return np.dot(Q[:,:i+1], y), i

In [188]:
#print cauchy_prec_gmres(A, b, M, it=it, tol=error)
print cauchy_left_prec_gmres(A, b, M, it=it, tol=error)

(array([ 3.91312584, -4.11006441,  2.16214429,  1.4818065 ]), 38)


/Library/Python/2.7/site-packages/ipykernel/__main__.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
/Library/Python/2.7/site-packages/ipykernel/__main__.py:26: ComplexWarning: Casting complex values to real discards the imaginary part
/Library/Python/2.7/site-packages/ipykernel/__main__.py:21: ComplexWarning: Casting complex values to real discards the imaginary part


In [189]:
print cauchy_prec_gmres(A, b, M, it=it, tol=error)

(array([-35.87693476,  14.81029692,  64.86003737, -40.37073375]), 33)


/Library/Python/2.7/site-packages/ipykernel/__main__.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
/Library/Python/2.7/site-packages/ipykernel/__main__.py:26: ComplexWarning: Casting complex values to real discards the imaginary part
/Library/Python/2.7/site-packages/ipykernel/__main__.py:21: ComplexWarning: Casting complex values to real discards the imaginary part
